<a href="https://colab.research.google.com/github/DavoodSZ1993/Dive-into-Deep-Learning-Notes-/blob/main/6_builderguide_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PyTorch Notes

* `torch.rand()`: Returns a tensor filled with random numbers from a uniform distribution on the interval [0, 1).

In [3]:
import torch

X = torch.rand(size=(2, 2))
X

tensor([[0.3017, 0.5035],
        [0.9197, 0.4851]])

## General Notes

### \*args and \**kwargs

* \*args: **Non-Keyword Arguments**: This is used in function definition is python and is used to pass a number of arguments to a function. 